In [4]:
# # Load model directly
# from transformers import AutoTokenizer, AutoModelForCausalLM

# tokenizer = AutoTokenizer.from_pretrained("openai-community/gpt2")
# model = AutoModelForCausalLM.from_pretrained("openai-community/gpt2")

In [3]:
from transformers import GPT2LMHeadModel
gpt2_hf = GPT2LMHeadModel.from_pretrained("gpt2") #initiating the init file of gpt2


/home/yamani.shirin/miniconda3/envs/llm/lib/python3.10/site-packages/transformers/utils/generic.py:311: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(
2024-09-23 10:41:56.936894: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-23 10:41:57.308516: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-23 10:41:57.434606: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-09-23 10:41:58.334581: I tensorflow/core/platform/cpu_feature_guard.cc:210] This

In [4]:
sd_gpt2 = gpt2_hf.state_dict()

for k,v in sd_gpt2.items():
    print(k,v.shape)

transformer.wte.weight torch.Size([50257, 768])
transformer.wpe.weight torch.Size([1024, 768])
transformer.h.0.ln_1.weight torch.Size([768])
transformer.h.0.ln_1.bias torch.Size([768])
transformer.h.0.attn.c_attn.weight torch.Size([768, 2304])
transformer.h.0.attn.c_attn.bias torch.Size([2304])
transformer.h.0.attn.c_proj.weight torch.Size([768, 768])
transformer.h.0.attn.c_proj.bias torch.Size([768])
transformer.h.0.ln_2.weight torch.Size([768])
transformer.h.0.ln_2.bias torch.Size([768])
transformer.h.0.mlp.c_fc.weight torch.Size([768, 3072])
transformer.h.0.mlp.c_fc.bias torch.Size([3072])
transformer.h.0.mlp.c_proj.weight torch.Size([3072, 768])
transformer.h.0.mlp.c_proj.bias torch.Size([768])
transformer.h.1.ln_1.weight torch.Size([768])
transformer.h.1.ln_1.bias torch.Size([768])
transformer.h.1.attn.c_attn.weight torch.Size([768, 2304])
transformer.h.1.attn.c_attn.bias torch.Size([2304])
transformer.h.1.attn.c_proj.weight torch.Size([768, 768])
transformer.h.1.attn.c_proj.bias 

In [5]:
sd_gpt2["transformer.h.0.ln_1.weight"].view(-1).max()

tensor(0.2527)

In [6]:
sd_gpt2["transformer.wte.weight"].view(-1)[:10]

tensor([-0.1101, -0.0393,  0.0331,  0.1338, -0.0485, -0.0789, -0.2398, -0.0895,
         0.0253, -0.1074])

In [5]:
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2024-09-21 13:27:55--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt’

input.txt           100%[===================>]   1.06M  --.-KB/s    in 0.1s    

2024-09-21 13:27:56 (11.0 MB/s) - ‘input.txt’ saved [1115394/1115394]



In [4]:
#read the file to inspect
with open(file='./data/input.txt', mode='r',  encoding='utf-8') as f:
    text = f.read()
    

In [5]:
text[:100]

'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou'

In [6]:
f'len text:{len(text)}'
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(f'number of unique chars:{chars}')
print(vocab_size)


number of unique chars:['\n', ' ', '!', '$', '&', "'", ',', '-', '.', '3', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']
65


In [7]:
#tokenization 
stoi = {ch:i for i, ch in enumerate(chars)}
itos = {i:ch for i,ch in enumerate(chars)}
#enocde
encode = lambda s: [stoi[c] for c in s] 
#decode
decode = lambda numz: ''.join([itos[i] for i in numz])

In [8]:
#encode('hi there!')
print(decode(encode('hi there!')))

hi there!


In [9]:
#encode entire data 
import torch
data = torch.tensor(encode(text), dtype= torch.long)

#same bc special case of having no dimension 
test = data.view(-1)[:1000] #self.long() is equivalent to self.to(torch.int64).
print(test.shape)

print(data.shape, data.dtype)
print(data[:100])


torch.Size([1000])
torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59])


In [10]:
#split to train/test
n = int(0.9 * len(data))
train_set = data[:n]
test_set = data[n:]

In [11]:
block_size = 8
train_set[:block_size+1]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [12]:
#data loader 
#time dimension
x = train_set[:block_size]
y = train_set[1:block_size + 1]

for t in range(block_size):
    context = x[:t+1] #to include t 
    target= y[t]
    print(f'when context is {context} we predict {target}')



when context is tensor([18]) we predict 47
when context is tensor([18, 47]) we predict 56
when context is tensor([18, 47, 56]) we predict 57
when context is tensor([18, 47, 56, 57]) we predict 58
when context is tensor([18, 47, 56, 57, 58]) we predict 1
when context is tensor([18, 47, 56, 57, 58,  1]) we predict 15
when context is tensor([18, 47, 56, 57, 58,  1, 15]) we predict 47
when context is tensor([18, 47, 56, 57, 58,  1, 15, 47]) we predict 58


In [13]:
#make use of prallalism; So a couple of same-size blocks 
#bTCH Dimension
torch.manual_seed(1337)
batch_size = 4  # independent sequences to be processed in parallel every forward/backward 
block_size = 8 #maximum context len in each of the sequence, can be less than block size! in each of the predictions

def get_batch(split):
    data = train_set if split == "train" else test_set
    
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i + block_size] for i in ix])
    y = torch.stack([data[i+1: i + block_size+1] for i in ix])
    return x, y

x_batch, y_b = get_batch(split='train')
print(f'input batch size is {x_batch.shape}')
print(f'the x_batch is: \n {x_batch}')

print('=='*20)

print(f'target y_batch size is {y_b.shape}')
print(f'the x_batch is: \n {y_b}')

print('=='*20)

for b in range(batch_size): #batch dimension
    for t in range(block_size): #time dimension
        context = x_batch[b, :t+1]
        target = y_b[b, t]
        print(f'when the context is {context.tolist()} we are predicting {target}')

# 32 independent exp packed in a single batch 


input batch size is torch.Size([4, 8])
the x_batch is: 
 tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])
target y_batch size is torch.Size([4, 8])
the x_batch is: 
 tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])
when the context is [24] we are predicting 43
when the context is [24, 43] we are predicting 58
when the context is [24, 43, 58] we are predicting 5
when the context is [24, 43, 58, 5] we are predicting 57
when the context is [24, 43, 58, 5, 57] we are predicting 1
when the context is [24, 43, 58, 5, 57, 1] we are predicting 46
when the context is [24, 43, 58, 5, 57, 1, 46] we are predicting 43
when the context is [24, 43, 58, 5, 57, 1, 46, 43] we are predicting 39
when the context is [44] we are predicting 53
when the context is [44, 53] 

In [14]:
print(x_batch) #input to the transformer 

tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])


In [17]:
#Baseline: bigram 
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)


class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        """ nn.Embedding is a wrapper
        input: list of indices.
        out: corresponding word embeddings
        """
        super().__init__()
        #wrapper 
        self.token_embeding_table = nn.Embedding(vocab_size, vocab_size) 
        
    def forward(self, idx, target=None):
        logits = self.token_embeding_table(idx) #(B,T, C)
        #print(logits.shape)
        if target is None:
            loss = None 
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C) #streched 
            # print(f'logits shape is {logits.shape} and it looks like:')
            # print(logits)
            target = target.view(B*T) #-1 streched 
            # print(f'logits shape is {target.shape} and it looks like:')
            # print(target)
            loss = F.cross_entropy(logits, target) #should be (B*T,C)  # LOSS: -ln(1/65)
            print(f'idx shape is {idx.shape}')
        return logits, loss
    
    
    def generate(self, idx, max_new_tokens):
        #idx shape is (B, T) which is the current context within some batch
        #so the job of generate is to take the (B,T) and extend it to (B, T + 1) in all the time dimensions within the batches
        for _ in range(max_new_tokens):
            #get predictions
            logits, loss = self(idx)
            logits = logits[:,-1,:] #becomes (B, C)
            
            probs = F.softmax(logits, dim=-1) #(B, C)
            #sample from this probs
            # If input is a matrix with m rows, out is an matrix of shape (m * num_samples)
            idx_next = torch.multinomial(probs, num_samples=1) # (B * 1) 
            #appending the sample to the current seq
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx 

In [19]:
#inspect forward pass     
obj = BigramLanguageModel(vocab_size)
logits, loss = obj(x_batch, y_b)
print(logits.shape)
print(loss) 


print(decode(obj.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=100)[0].tolist()))




idx shape is torch.Size([4, 8])
torch.Size([32, 65])
tensor(4.6630, grad_fn=<NllLossBackward0>)

hbH

:CLP.A!fq'3ggt!O!T?X!!SA?W&TrpvYybSE3w&S BXUhmiKYyTmWMPhhmnHKj!!btgnwNNULuEzRuYyiWEQxPX!$3C'MBj


In [28]:
#inspect nn.Embedding
input = torch.LongTensor([[1, 2, 4, 5], [4, 3, 2, 5]])
embed = nn.Embedding(6, 2)
print(embed(input))

tensor([[[ 0.5397,  1.1634],
         [ 0.6947, -1.2307],
         [-0.2909,  1.5433],
         [ 1.8641,  1.6099]],

        [[-0.2909,  1.5433],
         [ 0.6871,  1.1841],
         [ 0.6947, -1.2307],
         [ 1.8641,  1.6099]]], grad_fn=<EmbeddingBackward0>)
